# 数据格式化，看你的数据源格式,期望得到的格式如下

如果你的格式是：
{'content': 'xxxxxxx', 'summary': 'xxxxxxxxxxxxxxxxx}
{'content': 'xxxxxxx', 'summary': 'xxxxxxxxxxxxxxxxx}
下面的脚本可以帮你处理成：
{
    'conversations': [
                {'role': 'user', 'content': 'xxxxxxx'},
                {'role': 'assistant', 'content': 'xxxxxxxxxxxxxxxxx'}
    ]
}
这个格式可以区分不同的角色，也可以加入系统信息，同时，也可以进行多轮会话训练

In [ ]:
import json
from typing import Union
from pathlib import Path


def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    dir_name = _resolve_path(dir_name)
    if not dir_name.is_dir():
        dir_name.mkdir(parents=True, exist_ok=False)


def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    dct = json.loads(line)
                    sample = {'conversations': [{'role': 'user', 'content': dct['content']},
                                                {'role': 'assistant', 'content': dct['summary']}]}
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')

    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)

    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('data/ting', 'data/ting_fix')

# lora开始训练！
此处采用的是lora方式
lora可调整的参数就是r, lora_alpha, lora_dropout
分别是低维矩阵的秩（也就是降维矩阵的输出，升维矩阵的输入），lora和原网络的合并权重alpha，lora的dropout

peft_config:
  peft_type: LORA
  task_type: CAUSAL_LM
  r: 8
  lora_alpha: 32
  lora_dropout: 0.1

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python3 utils/finetune_hf.py  data/ting_fix  llm_weights/chatglm3-6b  configs/lora.yaml